In [1]:
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk import pos_tag


import json
from collections import defaultdict
import operator

import time
import math
import os
import sys
import numpy as np

In [2]:
directory="D:/Hacks/Wehack3/test data/test_sid"
namesofdoc=os.listdir(directory)


alpha=0.8
totaldocs=len(namesofdoc)

In [3]:
def read_data(file_name):
    with open(file_name,'r',encoding="utf8") as f:
        word_vocab = set() # not using list to avoid duplicate entry
        word2vector = {}
        for line in f:
            line_ = line.strip() #Remove white space
            words_Vec = line_.split()
            word_vocab.add(words_Vec[0])
            word2vector[words_Vec[0]] = np.array(words_Vec[1:],dtype=float)
    print("Total Words in DataSet:",len(word_vocab))
    return word_vocab,word2vector

In [4]:
totalvocab, w2v = read_data("D:/Deep learning/pretrainedmodels/Glove vectors/glove.6B.50d.txt")

Total Words in DataSet: 400000


In [5]:
w2v["dairy"]

array([ 0.057698, -0.51301 , -0.82971 ,  0.12021 , -0.011328,  1.3302  ,
       -1.2556  , -0.69629 ,  1.3744  , -0.099878,  0.574   , -0.3794  ,
        1.1517  , -0.78459 ,  0.28165 ,  0.73153 ,  0.26163 ,  1.3838  ,
        0.36276 , -0.27118 ,  1.0038  , -0.9954  ,  0.30433 ,  0.59716 ,
       -0.75876 , -0.79112 , -0.077779,  0.49264 ,  0.24474 ,  0.19268 ,
        1.5655  , -0.17366 ,  0.3011  ,  1.1682  ,  0.038647, -0.31717 ,
       -0.77932 ,  0.97368 ,  1.1681  ,  0.051528, -0.54678 , -0.69576 ,
        0.083017,  0.047535,  1.5585  ,  0.5962  , -1.2497  ,  0.18596 ,
        0.46701 , -0.20585 ])

In [6]:
#json_data=open("D:/Hacks/Wehack3/freqwords.json").read()

#freqwords = json.loads(json_data)


In [7]:
json_data=open("D:/Hacks/Wehack3/compresseddict.json").read()

compresseddict = json.loads(json_data)


In [48]:
q="taiwan"
q=word_tokenize(q)
q

['taiwan']

In [49]:
with open("D:/Hacks/Wehack3/vocab.txt", 'r',encoding='UTF-8') as f:
    vocab = [line.rstrip('\n') for line in f]

In [50]:
for i in range(0,len(q)):
    replace=""
    mindist=1000
    if q[i].lower() not in vocab:
        vec1=w2v[q[i]]
        for j in vocab:
            if j in totalvocab:
                vec2=w2v[j]
                dist = np.linalg.norm(vec1-vec2)
                if dist<mindist:
                    mindist=dist
                    replace=j
        q[i]=replace

In [51]:
x=pos_tag(q)
x

[('china', 'NN')]

In [52]:
dictofdoc={}

for i in namesofdoc:
    dictofdoc[i]=0

for i in x: 
    info=compresseddict[i[0].lower()]
    #finding number of documents query word is in
    numberofdocs=0
    numofdocsnocontext=0
    docs_counted=[]
    docs_nocontext_counted=[]
    for j in info:     #j has POStag,docn,number
        if j[0]==i[1]: #matches context
            if j[1] not in docs_counted: #to see whether or not the doc is already counted
                docs_counted.append(j[1])
                numberofdocs=numberofdocs+1
        else:
            if j[1] not in docs_counted: #to see whether or not the doc is already counted
                docs_nocontext_counted.append(j[1])
                numofdocsnocontext=numofdocsnocontext+1
            
            
    
    for j in info:     #j has POStag,docn,number
        if j[0]==i[1]: #matches context
            dictofdoc[j[1]]=dictofdoc[j[1]]+(alpha*j[2]*math.log10((totaldocs+1)/numberofdocs))
            
        else: #context not matched
            noncontext_match=j[2]
            dictofdoc[j[1]]=dictofdoc[j[1]]+((1-alpha)*j[2]*math.log10((totaldocs+1)/numofdocsnocontext))


In [53]:
sortedd = sorted(dictofdoc.items(), key=operator.itemgetter(1))
sortedd

[('ainme.txt', 0),
 ('choclate.txt', 0),
 ('hackathon.txt', 0),
 ('japan_stock.txt', 0),
 ('modi-japan-trip.txt', 0),
 ('modi.txt', 0),
 ('programming.txt', 0),
 ('usa.txt', 0),
 ('Japan.txt', 0.3999999999999999)]